<a href="https://colab.research.google.com/github/smit-patel-m/SummerInternship/blob/main/PUF_Random_forrest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer , classification_report, hamming_loss, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [4]:
import os
from google.colab import drive

if not os.path.exists("/content/db1"):
    drive.mount('/content/drive')
    !unzip "/content/drive/MyDrive/Summer Internship/ML Attack.zip" -d "/content/db1"
else:
    print("Folder '/content/db1' already exists. Skipping unzip.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/Summer Internship/ML Attack.zip
   creating: /content/db1/ML Attack/
  inflating: /content/db1/ML Attack/Responce_Delay_2us.xlsx  
  inflating: /content/db1/__MACOSX/ML Attack/._Responce_Delay_2us.xlsx  
  inflating: /content/db1/ML Attack/Challenge.xlsx  
  inflating: /content/db1/__MACOSX/ML Attack/._Challenge.xlsx  


# Load the Excel files into pandas DataFrames

In [5]:
# Load the Excel files into pandas DataFrames

response_delay_path = "/content/db1/ML Attack/Responce_Delay_2us.xlsx"
challenge_path = "/content/db1/ML Attack/Challenge.xlsx"
response_delay_sheet_names = pd.ExcelFile(response_delay_path).sheet_names
response_delay_df = pd.read_excel(response_delay_path, sheet_name=0,header=None)

challenge_df = pd.read_excel(challenge_path,header=None)


In [6]:
# Display the first few rows of each DataFrame to verify
print("Selected Sheet Name:", response_delay_sheet_names)

print("Response Delay DataFrame:")
display(response_delay_df.head())

print("\nChallenge DataFrame:")
display(challenge_df.head())

Selected Sheet Name: ['FD_MODE_1', 'VD_MODE_1', 'FD_MODE_2', 'VD_MODE_2', 'FD_MODE_3', 'VD_MODE_3', 'FD_MODE_4', 'VD_MODE_4', 'FD_MODE_5', 'VD_MODE_5', 'FD_MODE_6', 'VD_MODE_6', 'FD_MODE_7', 'VD_MODE_7', 'FD_MODE_8', 'VD_MODE_8']
Response Delay DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,1,1,1,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0,0,1,...,0,1,1,0,0,0,1,0,0,1
2,1,1,0,0,1,1,0,0,0,0,...,1,1,0,1,1,0,1,1,1,1
3,1,1,1,0,1,1,0,0,1,1,...,1,1,0,0,0,1,1,0,1,1
4,0,1,0,0,0,1,0,1,1,1,...,1,0,1,1,0,0,0,1,0,0



Challenge DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,0,0,0,0,0,1,1,1,...,1,0,1,1,0,0,0,1,1,1
1,0,1,1,0,0,1,0,1,1,1,...,0,1,1,0,1,0,0,1,0,0
2,0,0,1,0,1,0,0,1,0,0,...,0,0,1,1,0,1,1,0,1,1
3,0,0,0,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,1
4,1,1,1,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0


In [7]:
# Convert to NumPy
X = challenge_df.values.astype(np.float32)
Y = response_delay_df.values.astype(np.float32)


In [8]:
print("Input shape:", X.shape)
print("Output shape:", Y.shape)
assert X.shape[0] == Y.shape[0], "Mismatched input-output samples"


Input shape: (5000, 32)
Output shape: (5000, 32)


In [9]:

X_temp, X_val, Y_temp, Y_val = train_test_split(X, Y, test_size=0.10, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X_temp, Y_temp, test_size=2/9, random_state=42)


In [10]:

def exact_match_accuracy(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

In [11]:
history = []


for n_estimators in range(1, 35):  # 1 to 34 trees
    rf_base = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=None,
        min_samples_leaf=1,
        min_samples_split=2,
        max_features=None,
        random_state=42,
        n_jobs=-1
    )

    rf_model = MultiOutputClassifier(rf_base, n_jobs=-1)
    rf_model.fit(X_train, Y_train)

    # Predictions
    train_pred = rf_model.predict(X_train)
    val_pred = rf_model.predict(X_val)

    # Accuracy
    train_acc = accuracy_score(Y_train.flatten(), train_pred.flatten())
    val_acc = accuracy_score(Y_val.flatten(), val_pred.flatten())

    # Hamming Loss
    train_hamming = hamming_loss(Y_train, train_pred)
    val_hamming = hamming_loss(Y_val, val_pred)

    # Exact match accuracy
    val_exact = exact_match_accuracy(Y_val, val_pred)

    # Save to history
    history.append([
        n_estimators,
        train_acc,
        val_acc,
        train_hamming,
        val_hamming,
        val_exact
    ])

    # Print logs
    print(f"\n[Simulated Epoch: {n_estimators} Trees]")
    print(f" Train Accuracy (bitwise):   {train_acc:.4f}")
    print(f" Validation Accuracy:        {val_acc:.4f}")
    print(f" Train Hamming Loss:         {train_hamming:.4f}")
    print(f" Validation Hamming Loss:    {val_hamming:.4f}")
    print(f" Exact Match (Validation):   {val_exact:.4f}")

    if train_acc == 1.0:
        print(f"\n✅ Reached 100% training bitwise accuracy at {n_estimators} trees!")
        break




[Simulated Epoch: 1 Trees]
 Train Accuracy (bitwise):   0.8210
 Validation Accuracy:        0.5095
 Train Hamming Loss:         0.1790
 Validation Hamming Loss:    0.4905
 Exact Match (Validation):   0.0000

[Simulated Epoch: 2 Trees]
 Train Accuracy (bitwise):   0.8145
 Validation Accuracy:        0.4779
 Train Hamming Loss:         0.1855
 Validation Hamming Loss:    0.5221
 Exact Match (Validation):   0.0000

[Simulated Epoch: 3 Trees]
 Train Accuracy (bitwise):   0.9136
 Validation Accuracy:        0.5011
 Train Hamming Loss:         0.0864
 Validation Hamming Loss:    0.4989
 Exact Match (Validation):   0.0000

[Simulated Epoch: 4 Trees]
 Train Accuracy (bitwise):   0.9143
 Validation Accuracy:        0.4858
 Train Hamming Loss:         0.0857
 Validation Hamming Loss:    0.5142
 Exact Match (Validation):   0.0000

[Simulated Epoch: 5 Trees]
 Train Accuracy (bitwise):   0.9538
 Validation Accuracy:        0.5112
 Train Hamming Loss:         0.0462
 Validation Hamming Loss:    0.4

In [12]:
rf_train_pred = rf_model.predict(X_train)
rf_test_pred = rf_model.predict(X_test)


In [13]:

def exact_match_accuracy(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

print("\nRandom Forest Evaluation")
print(f" Train Accuracy:      {accuracy_score(Y_train, rf_train_pred):.4f}")
print(f" Train F1 Score:      {f1_score(Y_train, rf_train_pred, average='macro'):.4f}")
print(f" Train Hamming Loss:  {hamming_loss(Y_train, rf_train_pred):.4f}")

print(f"\n Test Accuracy:       {accuracy_score(Y_test, rf_test_pred):.4f}")
print(f" Test F1 Score:       {f1_score(Y_test, rf_test_pred, average='macro'):.4f}")
print(f" Test Hamming Loss:   {hamming_loss(Y_test, rf_test_pred):.4f}")
print(f"\n Exact Match Accuracy: {exact_match_accuracy(Y_test, rf_test_pred):.4f}")



Random Forest Evaluation
 Train Accuracy:      0.9986
 Train F1 Score:      1.0000
 Train Hamming Loss:  0.0000

 Test Accuracy:       0.0000
 Test F1 Score:       0.5773
 Test Hamming Loss:   0.4872

 Exact Match Accuracy: 0.0000


In [14]:
import joblib

def get_next_model_version(base_path="/content/drive/MyDrive/SummerIntern/Puf Models", prefix="puf_Random_Forest"):
    os.makedirs(base_path, exist_ok=True)
    existing = [f for f in os.listdir(base_path) if f.startswith(prefix)]
    versions = [int(re.findall(r'V(\d+)', name)[0]) for name in existing if re.search(r'V\d+', name)]
    next_version = max(versions) + 1 if versions else 1
    model_folder = f"{prefix}_V{next_version}"
    return os.path.join(base_path, model_folder)



def save_rf_model(model, path, history=None):
    os.makedirs(path, exist_ok=True)

    # Save model with joblib
    model_path = os.path.join(path, "model.joblib")
    joblib.dump(model, model_path)
    print(f"✅ Random Forest model saved at: {model_path}")

    # Save training history if provided
    if history is not None:
        history_df = pd.DataFrame(history, columns=[
            "Epoch (Trees)", "Train Accuracy", "Val Accuracy",
            "Train Hamming Loss", "Val Hamming Loss", "Val Exact Match"
        ])
        excel_path = os.path.join(path, "training_history.xlsx")
        history_df.to_excel(excel_path, index=False)
        print(f"📊 Training history saved at: {excel_path}")
def load_rf_model(path):
    model_path = os.path.join(path, "model.joblib")
    model = joblib.load(model_path)
    print(f"✅ Random Forest model loaded from: {model_path}")
    return model




In [16]:
save_path = get_next_model_version(
    base_path="/content/drive/MyDrive/SummerIntern/Puf Models",
    prefix="puf_Random_Forest"
)

save_rf_model(rf_model, save_path, history=history)



✅ Random Forest model saved at: /content/drive/MyDrive/SummerIntern/Puf Models/puf_Random_Forest_V1/model.joblib
📊 Training history saved at: /content/drive/MyDrive/SummerIntern/Puf Models/puf_Random_Forest_V1/training_history.xlsx


In [25]:

load_path = "/content/drive/MyDrive/SummerIntern/Puf Models/puf_Random_Forest_V1"
rf_model = load_rf_model(load_path)

# Evaluate
pred = rf_model.predict(X_test)
print("Test Accuracy:", 1 - hamming_loss(Y_test, pred))

✅ Random Forest model loaded from: /content/drive/MyDrive/SummerIntern/Puf Models/puf_Random_Forest_V1/model.joblib
Test Accuracy: 0.51284375


In [ ]:
def exact_match_accuracy(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

rf_test_pred = rf_model.predict(X_test)

print("Random Forest Exact Match Test Accuracy:", exact_match_accuracy(Y_test, rf_test_pred))
